Start



In [1]:
%load_ext autoreload
%autoreload 2

# Import some libraries

In [2]:

import os
import sys
import lmdb
import pickle
import numpy as np

# Cloning from github (Generative Models)



In [3]:
#!git clone https://github.com/kkahloots/Generative_Models.git # this is for loading git with correct brach



# Cloning from github (LMDB_Datasets)

In [4]:
#!cd Generative_Models
#!git clone https://github.com/azeghost/LMDB_Datasets # this is for loading git with correct brach

In [5]:
import os
print(os.getcwd())

C:\Users\Kriple\PycharmProjects\Generative_Models\notebooks\Super_Resolution\upscaling\Upsampling\ellwlb\CBSD68


In [6]:
sep_local = os.path.sep
#os.chdir('..'+sep_local+'..'+sep_local+'..'+sep_local+'..'+sep_local+'..'+sep_local+'..'+sep_local)
os.chdir('C:/Users/Kriple/PycharmProjects/Generative_Models')
print(os.getcwd())

C:\Users\Kriple\PycharmProjects\Generative_Models


# LMDB transformer

In [7]:
#!pip install colorlog

In [8]:
from colorlog import ColoredFormatter




**LMDB Transformer **

In [9]:
#!cp -r LMDB_Datasets/transformation/ Generative_Models/

In [10]:
#import os
#os.chdir('Generative_Models')
#os.getcwd()

In [11]:
import tensorflow as tf

In [12]:
image_size_original=(481, 321, 3)
scale = 3
image_size = list(map(lambda x: x//scale , image_size_original[:-1])) + [image_size_original[-1]]
image_size = (*image_size,)

In [13]:
image_size

(160, 107, 3)

In [14]:
inputs_shape = list(map(lambda x: x//2 , image_size[:-1])) + [image_size[-1]]
inputs_shape = (*inputs_shape,)

In [15]:
import cv2
import numpy as np

In [16]:
small_size = tuple([a//3 for a in image_size_original][:-1])

In [17]:
shrink_fn = lambda image: np.asarray(cv2.resize(image, dsize=small_size, interpolation=cv2.INTER_CUBIC))

In [18]:
images_dir = 'C:/Users/Kriple/PycharmProjects/Generative_Models/data/CBSD68' #Folder to images ( without name of the folder which we moved images before)
validation_percentage = 30
valid_format = 'png'

In [19]:
from transformation.lmdb_transformer import SRLmdbTransformer

In [20]:
transformer = SRLmdbTransformer(image_dir=images_dir, trans_func=shrink_fn,
                              validation_pct = validation_percentage, valid_image_formats = valid_format)

# Evaluating numbers of training and validation sets

In [21]:

val_num_images = 0
tra_num_images = 0
for key in transformer.image_lists.keys():
    tra_num_images += len(transformer.image_lists[key]['training'])
    val_num_images += len(transformer.image_lists[key]['validation'])
    
print(tra_num_images, val_num_images)

48 20


# Getting labels by filename

In [22]:
from transformation.transformative_custom_dataset import get_label_by_filename


In [23]:
def get_label_by_filename(img_path):
    name, _ = os.path.splitext(img_path)
    vid_img_arr = name.split(sep=os.sep)[-2:]
    return {'dir': (vid_img_arr)[0], 'filename': (vid_img_arr)[1]}

# Transforming images to LMDB

In [24]:
lmdb_dir = 'C:/Users/Kriple/PycharmProjects/Generative_Models/data/.CBSD68_low2high_LMDB'
lmdb_dir

'C:/Users/Kriple/PycharmProjects/Generative_Models/data/.CBSD68_low2high_LMDB'

In [25]:
#!mv data/.CBSD68/02 data/.CBSD68/a

In [26]:
image_size_original[:-1]

(481, 321)

In [27]:
transformer.transform_store(labels_fn=get_label_by_filename, image_dir=images_dir, lmdb_dir=lmdb_dir,
           category='training', target_size=image_size_original[:-1], color_mode='rgb')


Total number of imgs for catagory 48
Storing 48 C:/Users/Kriple/PycharmProjects/Generative_Models/data/.CBSD68_low2high_LMDB\ into _training from folder a


In [28]:
[a//3 for a in image_size_original][:-1]

[160, 107]

In [29]:
image_size_original[:-1]

(481, 321)

In [30]:
transformer.transform_store(labels_fn=get_label_by_filename, image_dir=images_dir, lmdb_dir=lmdb_dir
           ,category='validation', target_size=image_size_original[:-1], color_mode='rgb')

Total number of imgs for catagory 20
Storing 20 C:/Users/Kriple/PycharmProjects/Generative_Models/data/.CBSD68_low2high_LMDB\ into _validation from folder a


# Saving meta data of dataset

In [31]:
#!touch C:/Users/Kriple/PycharmProjects/Generative_Models/data/.CBSD68_low2high_LMDB/meta_info.json #creating a json file

In [32]:
!ls C:/Users/Kriple/PycharmProjects/Generative_Models/data/.CBSD68_low2high_LMDB

_training
_validation
meta_info.json


In [33]:

info = {
    'Dataset_name': "CBSD68_low2high",
    'Dataset_size': val_num_images + tra_num_images,
    'val_num_images':val_num_images,
    'tra_num_images' : tra_num_images
}

transformer.save_metadata(lmdb_dir, info)

In [34]:
#transformer.get_metadata(lmdb_dir) #checking whether it is working or not

In [35]:
!ls -l C:/Users/Kriple/PycharmProjects/Generative_Models/data/.CBSD68_low2high_LMDB/

total 1
drwxr-xr-x 1 Kriple None   0 Nov  4 20:17 _training
drwxr-xr-x 1 Kriple None   0 Nov  4 20:18 _validation
-rw-r--r-- 1 Kriple None 111 Nov  4 20:19 meta_info.json


In [36]:
xxx

NameError: name 'xxx' is not defined

# I tried to read LMDB, but..

In [ ]:
import lmdb
import pickle

def read_many_lmdb(lmdb_dir, num_images):

    images, labels = [], {}
    env = lmdb.open(lmdb_dir, readonly=True)

    # Start a new read transaction
    with env.begin() as txn:
        # Read all images in one single transaction, with one lock
        # We could split this up into multiple transactions if needed
        for image_id in range(num_images):
            data = txn.get(f"{image_id:08}".encode("ascii"))
            # Remember that it's a CIFAR_Image object 
            # that is stored as the value
            dataset = pickle.loads(data)
            images.append(dataset.get_image())
            
            labels_list = [attr for attr in dir(dataset) if not callable(getattr(dataset, attr)) and (not attr.startswith("__")) and 
                           (not attr in ['image','channels',  'size'] )]

            for label in labels_list:
                # _lab = {label: eval(f'dataset.{label}')}
#                 print({label: eval(f'dataset.{label}')})
                # labels = {**labels, **_lab}
                if label in labels:
                    labels[label].append(eval(f'dataset.{label}'))
                else:
                    labels = {label: [eval(f'dataset.{label}')] }
                
    env.close()
    return {'images': images, **labels}

I don't know why it gives the following error.

In [ ]:
ds = read_many_lmdb('/content/Generative_Models/CBSD68_LMDB/_training', 5)

# Zipping

In [ ]:
!ls data/.CBSD68_low2high_LMDB/_training/

In [ ]:
!tar -cvjf CBSD68_low2high_LMDB.tar.bz data/.CBSD68_low2high_LMDB

In [ ]:
!ls CBSD68_low2high_LMDB.tar.bz -l

In [ ]:
!zip -r LMDB_Datasets.zip  LMDB_Datasets/

In [ ]:
!ls

In [ ]:
!tar -cvjf LMDB_Datasets.tar.bz LMDB_Datasets